# 1. Preparing dataset

We need to prepare a dataset from the table from wikipedia 'Toronto - FSAs' - a Toronto neighbourhoods/ZIP codes table

First let download a .csv file

In [2]:
# import pandas, numpy

import pandas as pd
import numpy as np

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
#dataset with Toronto postal codes have been imported as 'body' from IBM Watson assets storage in invisible cell above

df = pd.read_csv(body)
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
df.shape

(288, 3)

Now we will get rid of Not Assigned values in 'Borough' column and change this value in Neighbourhood with the correspoding Borough

In [6]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop = True, inplace = True)

df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
# Get index of 'Not assigned' values in Neighbours

df.index[df.Neighbourhood =='Not assigned'].tolist()

[6]

In [8]:
#Change them with corresponding Borough names

changename = df.iloc[6,1]
df = df.replace(to_replace = 'Not assigned', value = changename)
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [9]:
df.shape

(211, 3)

# 2. Merging Borough names and geolocation info

Let's import new table with coordinates for the postal codes

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
df_geo = pd.read_csv(body)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#Merge both dataframes based on Postal codes values

df_geo = df_geo.rename(columns = {'Postal Code' : 'Postcode'})
df_neigh = pd.merge(df, df_geo, on = 'Postcode')
df_neigh.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


# 3. Research

In [13]:
# get all boroughs names

df_neigh.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Let's select neighbourhoods located only in Toronto area (boroughs contaning word 'Toronto')

In [21]:
df_neigh_tor = df_neigh.loc[df_neigh['Borough'].str.contains('Toronto')].reset_index(drop = True)
df_neigh_tor.head(8)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [22]:
df_neigh_tor.shape

(74, 5)

### Create a map of Toronto with neighborhoods superimposed on top.

In [17]:
!pip install folium
import folium

!pip install geopy
from geopy.geocoders import Nominatim

print('Folium installed and imported')

     |████████████████████████████████| 92kB 16.7MB/s eta 0:00:01
Folium installed and imported


In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="torontox")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neigh_tor['Latitude'], df_neigh_tor['Longitude'], df_neigh_tor['Borough'], df_neigh_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [28]:
# The code was removed by Watson Studio for sharing.